In [2]:
import numpy as np

BOARD_ROWS = 3
BOARD_COLS = 4 #4x3 영역
WIN_STATE = (0, 3) #보상이 증가하는 중단 상태 위치
LOSE_STATE = (1, 3) #보상이 감소하는 중단 상태 위치
BLOCKED_STATE = (1, 1)
START = (2, 0) #시작 상태 위치
DETERMINISTIC = False #상태전이 함수 활성화

#what is state?
class State:
    def __init__(self, state = START):
        self.state = state
        self.isEnd = False
        self.determine = DETERMINISTIC
#이기는 상태일 경우 1반환, 지는 상태일 경우 -1 반환
    def giveReward(self):
        if self.state == WIN_STATE:
            return 1
        elif self.state == LOSE_STATE:
            return -1
        else:
            return 0 #그 외는 0 반환

    def isEndFunc(self): #지거나 이기는 상태라면 끝 상태가 사실.
        if (self.state == WIN_STATE) or (self.state == LOSE_STATE):
            self.isEnd = True
#상하좌우 중 하나를 선택 함. 딘,각 선택 후 확률적으로 다른 액션을 취하게 될 확률이 존재함.
    def _chooseActionProb(self, action):
        if action == "U":
            return np.random.choice(["U", "L", "R"], p = [0.8, 0.1, 0.1]) #0.2의 확률로 l이나 r로 튐.
        if action == "D":
            return np.random.choice(["D", "L", "R"], p = [0.8, 0.1, 0.1])
        if action == "L":
            return np.random.choice(["L", "U", "D"], p = [0.8, 0.1, 0.1])
        if action == "R":
            return np.random.choice(["R", "U", "D"], p = [0.8, 0.1, 0.1])
    #액션 선택에 따른 결과를 반영해주는 기능적 함수
    def nxtPosition(self, action):
        if self.determine:
            if action == "U":
                nxtState = (self.state[0] - 1, self.state[1])
            elif action == "D":
                nxtState = (self.state[0] + 1, self.state[1])
            elif action == "L":
                nxtState = (self.state[0], self.state[1] - 1)
            else:
                nxtState = (self.state[0], self.state[1] + 1)
            self.determine = False
        else:
          #상태전이함수를 적용함
            action = self._chooseActionProb(action)
            self.determine = True
            nxtState = self.nxtPosition(action)

        if (nxtState[0] >= 0) and (nxtState[0] <= 2):
            if (nxtState[1] >= 0) and (nxtState[1] <= 3):
                if nxtState != BLOCKED_STATE:
                    return nxtState
                    #벽관련된 부분임 크게 안중요함.
        return self.state

#에이전트 수행에 관한 내용
class Agent:

    def __init__(self):
        self.states = []
        self.actions = ["U", "D", "L", "R"]
        self.State = State()
        self.isEnd = self.State.isEnd
        self.lr = 0.2
        self.decay_gamma = 0.9 #무한 왔다갔다 방지용 시간에 따른 할인율 설정.

      #격자공간 상 각 위치들에 대한 Q값 초기화
        self.Q_values = {}
        for i in range(BOARD_ROWS):
            for j in range(BOARD_COLS):
                self.Q_values[(i, j)] = {}
                for a in self.actions:
                    self.Q_values[(i, j)][a] = 0

    def chooseAction(self):
        max_nxt_reward = 0
        action = ""

        for a in self.actions:
            current_position = self.State.state
            nxt_reward = self.Q_values[current_position][a]
            if nxt_reward >= max_nxt_reward:
                action = a
                max_nxt_reward = nxt_reward #맥스 다음 리와드 보다 특정 상태에서 특정 액션을 취해 얻을 수 있는 리와트가 크면 반영하고 이에 맞춰 변경해주는 코드.
        return action

    def takeAction(self, action):#액션이 상태에 반영되게 해주는 코드.
        position = self.State.nxtPosition(action)
        return State(state = position)

    def reset(self):
        self.states = []
        self.State = State()
        self.isEnd = self.State.isEnd #상태가 isend면 초기화

#이걸 에피소드 개수만큼 반복, 여기서는 10.
    def play(self, episodes = 10):
        i = 0
        while i < episodes:
            if self.State.isEnd:
                reward = self.State.giveReward()
                for a in self.actions:
                    self.Q_values[self.State.state][a] = reward
                for s in reversed(self.states):
                    current_q_value = self.Q_values[s[0]][s[1]]
                    reward = current_q_value + self.lr * (self.decay_gamma * reward - current_q_value)
                    self.Q_values[s[0]][s[1]] = round(reward, 3)
                self.reset()
                i += 1
            else:
                action = self.chooseAction()
                self.states.append([(self.State.state), action])
                self.State = self.takeAction(action)
                self.State.isEndFunc()
                self.isEnd = self.State.isEnd

ag = Agent()

ag.play(1000)
print("latest Q-values ... \n")
print(ag.Q_values)

latest Q-values ... 

{(0, 0): {'U': 0, 'D': 0, 'L': 0, 'R': 0.609}, (0, 1): {'U': 0, 'D': 0, 'L': 0, 'R': 0.754}, (0, 2): {'U': 0, 'D': 0, 'L': 0, 'R': 0.862}, (0, 3): {'U': 1, 'D': 1, 'L': 1, 'R': 1}, (1, 0): {'U': 0, 'D': 0, 'L': 0, 'R': 0.172}, (1, 1): {'U': 0, 'D': 0, 'L': 0, 'R': 0}, (1, 2): {'U': 0, 'D': 0, 'L': 0.497, 'R': -0.18}, (1, 3): {'U': -1, 'D': -1, 'L': -1, 'R': -1}, (2, 0): {'U': 0, 'D': 0, 'L': 0.078, 'R': -0.001}, (2, 1): {'U': 0, 'D': 0, 'L': 0.077, 'R': -0.005}, (2, 2): {'U': 0, 'D': 0, 'L': 0.121, 'R': -0.03}, (2, 3): {'U': 0, 'D': 0, 'L': 0, 'R': -0.166}}
